# Overview

For the following datasets:

1. https://universe.roboflow.com/eyedetection/iris-detection
2. https://universe.roboflow.com/eyedetection/eye-detection-fnwhr

and create a dataset suitable for feeding to an autoencoder via the following steps:

1. Download the dataset from roboflow, if not already downloaded
2. For each image in the dataset
    1. Identify bounding box groups that label eyes (there may be more than one, at least in the eye-detection-fnwhr dataset
    1. For each bounding box group, derive the bounding box that encompasses all of the eye-part bounding boxes, in order to capture the whole eye
    1. Use that encompassing bounding box to crop out an eye image
    1. Write the eye image to a file in the target directory
        
* Milestone 1: get it working for iris detection, which has single bounding box group and better featured images.
* Milestone 2: get it working for eye-detection-fnwhr, which has multiple bounding box groups

# Python imports

In [1]:
from roboflow import Roboflow
from pycocotools.coco import COCO
import os
import csv
import helper_functions

# Extract target images

Note that coco bounding boxes are in [top-left-x, top-left-y, width, height] format ([x_min, y_min, width, height])


In [2]:
coco_train_annotation_path = os.path.join(os.getcwd(), "Eye-Detection-3/train", "_annotations.coco.json")
coco_train = COCO(annotation_file=coco_train_annotation_path)
coco_train

loading annotations into memory...
Done (t=0.15s)
creating index...
index created!


In [4]:
# Category IDs.
cat_ids = coco_train.getCatIds()

In [5]:
super_bboxes_and_filenames = helper_functions.eye_dataset_extract_super_bounding_boxes(coco_train, cat_ids)

In [6]:
target_dir_path = "eye_dataset_autoencoder_training_files"
if not os.path.exists(target_dir_path):
    os.makedirs(target_dir_path)
    
image_file_names = helper_functions.eye_dataset_crop_and_write_super_bboxes(target_dir_path, super_bboxes_and_filenames)

with open(os.path.join(target_dir_path, "annotations.csv"), "w") as f:
    writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)

    for image_file_name in image_file_names:
        writer.writerow([image_file_name])

print(f"Wrote {len(image_file_names)} image file names to annotations.csv")

Wrote 2480 cropped images to eye_dataset_autoencoder_training_files
Wrote 2480 image file names to annotations.csv
